In [1]:
import os

import pandas as pd
import numpy as np

# Coppied from sample main
local_path = '/home/milosh-dr/code/MPs'
files = os.listdir(local_path)
# 
results = []
dfs = []
for filename in files:
    if filename.startswith('results'):
        results.append(filename)

for filename in sorted(results):
    df = pd.read_csv(filename)
    dfs.append(df)

df = pd.concat(dfs, axis=1)

In [2]:
df.head()

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32,51/33
0,PiS,Adamczyk Andrzej,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
1,PiS,Andruszkiewicz Adam,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
2,PiS,Andzel Waldemar,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
3,PiS,Ardanowski Jan Krzysztof,Przeciw,Za,Przeciw,Przeciw,Za,Nieobecny,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN
4,PiS,Arent Iwona,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw,Przeciw,Przeciw,...,Za,Przeciw,Przeciw,Przeciw,Za,Za,Przeciw,Za,Za,NaN


Getting rid of incorrectly parsed data

In [3]:
nans = df.isna().sum()
nans[nans>0]

59/244     24
51/33     460
dtype: int64

In [4]:
df.drop(['59/244', '51/33'], axis=1, inplace=True)

Coding string descriptions of votes into numeric values. Marking absences with np.nans for now

In [5]:
mapping = {}
values = df.iloc[:,5].value_counts().index.tolist()
new_values = [0, 1 , np.nan, .5]
for value, new_value in zip(values, new_values):
    mapping[value] = new_value
mapping

{'Przeciw': 0, 'Za': 1, 'Nieobecny': nan, 'Wstrzymał się': 0.5}

In [6]:
for col in df.columns[2:]:
    df[col] = df[col].map(mapping)

There might be some meaningful absences, when MPs actually expressed their disagreement, but one need to be very careful replacing those values. Let's explore this. 

In [7]:
# Let's sum all the absences across the parties
not_present = df.groupby('Party').apply(lambda x: x.isna().sum())
not_present

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/23,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32
Party,,,,,,,,,,,,,,,,,,,,,
KO,0,0,5,5,5,5,5,12,9,5,...,0,0,0,0,0,0,0,0,0,0
KP,0,0,1,1,1,1,1,4,1,2,...,1,1,1,1,1,1,1,1,1,1
Konfederacja,0,0,2,2,2,2,2,6,0,0,...,0,0,0,0,0,0,0,0,0,0
Kukiz15,0,0,1,1,1,1,1,1,3,1,...,0,0,1,1,0,0,0,0,1,0
Lewica,0,0,5,4,4,4,4,4,7,3,...,0,0,0,0,0,0,0,0,0,0
PPS,0,0,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PiS,0,0,9,8,11,8,7,12,7,7,...,0,2,0,3,0,1,2,1,2,1
Polska2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Computing party sizes
party_sizes = df[['Party', 'MPS']].groupby('Party').agg(len)['MPS']
not_present['party_size'] = party_sizes

In [9]:
def absence_checker(row):
    """As a rule of thumb we decide to take a closer look whenever more then 75% of party members were absent.
    Considering only parties having more then 5 members"""
    
    if row['party_size'] < 6:
        row[:]=np.nan
        return row
    
    row[~(row>.75*row['party_size'])] = np.nan
    return row


meaningful_absence = not_present.apply(absence_checker, axis=1)
subset = meaningful_absence.columns[1:-1].tolist()
meaningful_absence = (meaningful_absence
                      .dropna(axis=0, how='all', subset=subset)
                      .dropna(axis=1, how='all')
                      .drop('party_size', axis=1))
meaningful_absence

,55/62
Party,
KO,109.0


Since there was only one situation meeting our criteria and thus revealing strong probability of meaningful absence I'll imput those nans with 0.

In [11]:
df.loc[(df['Party']=='KO'), '55/62'] = df.loc[(df['Party']=='KO'), '55/62'].fillna(0)

In [13]:
df.loc[(df['Party']=='KO'), '55/62']

228    0.0
229    0.0
230    0.0
231    0.5
232    0.0
      ... 
349    0.0
350    0.0
351    0.0
352    0.0
353    0.0
Name: 55/62, Length: 126, dtype: float64

In case of this analysis it's better to assume that in general MPs that are absent on a particular vote normally would vote with majority of the party they belong to. It wouldn't be reasonable to imput anomalies ourselves if we search for those.

In [14]:
df = df.groupby('Party', group_keys=False).apply(lambda x: x.fillna(round(x.mean(numeric_only=True))))

In [15]:
df.isna().sum()[df.isna().sum()>0]

60/2     4
57/1     3
53/58    4
dtype: int64

In [16]:
df.loc[df.isna().any(axis=1), ['MPS', 'Party', '60/2', '57/1', '53/58']]

,MPS,Party,60/2,57/1,53/58
446,Kukiz Paweł,Kukiz15,NaN,1.0,NaN
447,Sachajko Jarosław,Kukiz15,NaN,1.0,NaN
448,Tyszka Stanisław,Kukiz15,NaN,1.0,NaN
449,Żuk Stanisław,Kukiz15,NaN,1.0,NaN
453,Girzyński Zbigniew,PS,0.0,NaN,0.0
454,Sośnierz Andrzej,PS,0.0,NaN,0.0
455,Ścigaj Agnieszka,PS,0.0,NaN,0.0


We are left with just a few missing values indicating absence of the entire party (we didn't include those small parties (3-4 members) in the code above). Let's replace those with 0.5 indicating no preference. It seems to be safer choice than arbitrary setting the preference to 0 or 1.

In [17]:
for col in ['60/2', '57/1', '53/58']:
    df[col] = df[col].fillna(0.5)

In [18]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

In [19]:
df.head(15)

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6,62/7,62/8,...,51/23,51/24,51/25,51/26,51/27,51/28,51/29,51/30,51/31,51/32
0,PiS,Adamczyk Andrzej,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1,PiS,Andruszkiewicz Adam,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
2,PiS,Andzel Waldemar,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,PiS,Ardanowski Jan Krzysztof,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,PiS,Arent Iwona,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
5,PiS,Ast Marek,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
6,PiS,Babalski Zbigniew,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
7,PiS,Babinetz Piotr,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
8,PiS,Bartosik Ryszard,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
9,PiS,Bartuś Barbara,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0


In [20]:
df.Party.unique()

array(['PiS', 'KO', 'Lewica', 'KP', 'Konfederacja', 'Polska2050',
       'Porozumienie', 'Kukiz15', 'PPS', 'PS', 'niez.'], dtype=object)

## Clustering

In [21]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN

In [23]:
km = KMeans(n_clusters=11)
ms = MeanShift()
db = DBSCAN(eps=1)

clustering_data=[df[['MPS', 'Party']]]
for model in (km, ms, db):
    labels = pd.Series(model.fit_predict(df.iloc[:,2:]), name='clusters_{}'.format(model.__class__.__name__))
    clustering_data.append(labels)
clusters = pd.concat(clustering_data, axis=1)
clusters

,MPS,Party,clusters_KMeans,clusters_MeanShift,clusters_DBSCAN
0,Adamczyk Andrzej,PiS,0,0,-1
1,Andruszkiewicz Adam,PiS,0,0,-1
2,Andzel Waldemar,PiS,0,0,0
3,Ardanowski Jan Krzysztof,PiS,0,0,0
4,Arent Iwona,PiS,0,0,0
...,...,...,...,...,...
455,Ścigaj Agnieszka,PS,5,7,-1
456,Ajchler Zbigniew,niez.,5,12,-1
457,Galla Ryszard,niez.,7,11,-1
458,Mejza Łukasz,niez.,5,9,-1


In [24]:
cols = [x for x in clusters.columns if x.startswith('clusters')]
for col in cols:
    display(clusters.groupby(col).apply(lambda x: x['Party'].value_counts()))

clusters_KMeans              
0                PiS             228
1                KO               41
2                Konfederacja     11
3                Lewica           44
4                KP               24
                 Porozumienie      5
5                PS                3
                 niez.             2
6                PPS               3
7                niez.             2
8                Kukiz15           4
9                Polska2050        8
10               KO               85
Name: Party, dtype: int64

clusters_MeanShift              
0                   PiS             228
1                   KO              126
2                   Lewica           44
3                   KP               24
                    Porozumienie      5
4                   Polska2050        8
5                   Konfederacja      7
6                   Kukiz15           3
7                   PS                2
8                   PPS               2
9                   niez.             1
10                  PS                1
11                  niez.             1
12                  niez.             1
13                  Kukiz15           1
14                  PPS               1
15                  Konfederacja      1
16                  Konfederacja      1
17                  Konfederacja      1
18                  Konfederacja      1
19                  niez.             1
Name: Party, dtype: int64

clusters_DBSCAN              
-1               KO              114
                 PiS              81
                 Lewica           44
                 KP               24
                 Konfederacja     11
                 Polska2050        8
                 Porozumienie      5
                 Kukiz15           4
                 niez.             4
                 PPS               3
                 PS                3
 0               PiS             131
 1               PiS              16
 2               KO                7
 3               KO                5
Name: Party, dtype: int64

## Dimensionality reduction

Let's explore the relevance of particular votes.

In [25]:
votes_info = pd.read_csv('votes_info.csv')
votes_info.head()

,session_no,session_url,date,vote_no,vote_url,vote_time,vote_topic,vote_type
0,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,1,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:39:39,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawki nr 1-7 i 9
1,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,2,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:40:10,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawka 8
2,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,3,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:40:33,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawki nr 10-11
3,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,4,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:41:31,Pkt. 2 Pierwsze czytanie poselskiego projektu ...,wniosek o odrzucenie projektu w pierwszym czyt...
4,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,5,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:42:25,62. posiedzenie Sejmu Rzeczypospolitej Polskie...,wniosek o zastosowanie art. 39 ust. 3 regulami...


A brief look into the vote_type column makes it clear that we can easily group votes into several major categories.

In [84]:
mapping = {
    'odrzuce': 'odrzucenie',
    'poprawk': 'poprawki',
    'głosowanie nad całością': 'głosowanie za',
    'mniejszości': 'wniosek mniejszości',
    'odpowiedzialności karnej': 'odpowiedzialność karna',
    'inne': 'inne'
}
vote_category = (votes_info.vote_type
     .str.extract('(odrzuce|poprawk|głosowanie nad całością|mniejszości|odpowiedzialności karnej)', expand=False)
     .fillna('inne')
     .map(mapping))

In [85]:
vote_category.value_counts()

poprawki                  1220
głosowanie za              173
inne                       169
wniosek mniejszości         85
odrzucenie                  61
odpowiedzialność karna       6
Name: vote_type, dtype: int64

In [86]:
votes_info['vote_category'] = vote_category

As we can see the majority of votes refer to the law alterations, whereas the actual vote for or against the complete project happened around 230 times. Let's select those vote ids for our consideration. Those votes might better mirror shape of MPs preferences. We can also expect from it to distill our df to slightly less noisy version.

In [118]:
selection = ['głosowanie za', 'odrzucenie', 'odpowiedzialność karna']
selected_votes = votes_info[votes_info['vote_category'].isin(selection)]
vote_ids = (selected_votes['session_no'].astype('str')+'/'+selected_votes['vote_no'].astype('str')).tolist()

In [119]:
new_cols = ['MPS', 'Party']
new_cols.extend(vote_ids)

In [126]:
mask = df.columns.isin(new_cols)
df_short = df[df.columns[mask]]
df_short

,Party,MPS,62/4,62/9,62/10,62/16,62/17,62/22,62/24,62/26,...,52/77,52/114,51/3,51/4,51/5,51/6,51/13,51/16,51/17,51/21
0,PiS,Adamczyk Andrzej,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,PiS,Andruszkiewicz Adam,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,PiS,Andzel Waldemar,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,PiS,Ardanowski Jan Krzysztof,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
4,PiS,Arent Iwona,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,PS,Ścigaj Agnieszka,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
456,niez.,Ajchler Zbigniew,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
457,niez.,Galla Ryszard,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
458,niez.,Mejza Łukasz,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.5,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.5


Second approach to clustering. Next up: better parameter tunning and evaluation

In [127]:
km = KMeans(n_clusters=11)
ms = MeanShift()
db = DBSCAN(eps=1)

clustering_data=[df[['MPS', 'Party']]]
for model in (km, ms, db):
    labels = pd.Series(model.fit_predict(df_short.iloc[:,2:]), name='clusters_{}'.format(model.__class__.__name__))
    clustering_data.append(labels)
clusters = pd.concat(clustering_data, axis=1)
clusters

,MPS,Party,clusters_KMeans,clusters_MeanShift,clusters_DBSCAN
0,Adamczyk Andrzej,PiS,0,0,0
1,Andruszkiewicz Adam,PiS,0,0,0
2,Andzel Waldemar,PiS,0,0,0
3,Ardanowski Jan Krzysztof,PiS,0,0,0
4,Arent Iwona,PiS,0,0,0
...,...,...,...,...,...
455,Ścigaj Agnieszka,PS,3,10,-1
456,Ajchler Zbigniew,niez.,3,25,-1
457,Galla Ryszard,niez.,8,23,-1
458,Mejza Łukasz,niez.,0,21,-1


In [128]:
cols = [x for x in clusters.columns if x.startswith('clusters')]
for col in cols:
    display(clusters.groupby(col).apply(lambda x: x['Party'].value_counts()))

clusters_KMeans              
0                PiS             228
                 niez.             1
1                KO              126
2                Konfederacja     11
3                PS                3
                 niez.             1
4                Lewica           27
5                Porozumienie      5
6                KP               24
7                Polska2050        8
8                niez.             2
9                Lewica           17
                 PPS               3
10               Kukiz15           4
Name: Party, dtype: int64

clusters_MeanShift              
0                   PiS             227
1                   KO              122
2                   Lewica           44
3                   KP               24
4                   Polska2050        8
5                   Porozumienie      5
6                   Konfederacja      4
7                   KO                3
8                   Konfederacja      2
9                   Kukiz15           2
10                  PS                2
11                  PPS               1
12                  PPS               1
13                  PPS               1
14                  KO                1
15                  Konfederacja      1
16                  Konfederacja      1
17                  Konfederacja      1
18                  Konfederacja      1
19                  Konfederacja      1
20                  niez.             1
21                  niez.             1
22                  PiS               1
23                  niez.             1
24     

clusters_DBSCAN              
-1               KO               20
                 Konfederacja     11
                 Lewica            8
                 PiS               7
                 Porozumienie      5
                 KP                4
                 Kukiz15           4
                 niez.             4
                 PPS               3
                 PS                3
                 Polska2050        1
 0               PiS             221
 1               KO              106
 2               Lewica           36
 3               KP               20
 4               Polska2050        7
Name: Party, dtype: int64